# SqueezeNet SE Auto

In [1]:
from __future__ import absolute_import, division, print_function

import itertools
import glob, os

import matplotlib.pylab as plt
import numpy as np


import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

from tensorflow import keras
import cv2

import tensorflow.keras.backend as K
from keras_tuner import HyperParameters
from tensorflow.keras import layers
from keras_tuner import RandomSearch,BayesianOptimization, Hyperband
from tensorflow import keras

import neptunecontrib.monitoring.kerastuner as npt_utils
from keras_tuner import HyperParameters, Objective

from keras_tuner.engine import hypermodel

from model_all import SqueezeNetModel, SqueezeNet11Model,SqueezeNetSEAutoModel
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")



source_dir = '../TsinghuaFEDImages'
data_dir = '../TsinghuaFED_class' # landmarks of each subject_emotion

emotion_class = ['fear','neutral','surprise','anger','content','disgust','sad','happy']

IMG_WIDTH = 224
IMG_HEIGHT = 224
nb_classes = len(emotion_class)
ratio = 1.0
EPOCH = 200

def load_dataset(data_folder, img_dir):
   

  img_list = data_folder+'/train.csv'

  f = open(img_list)
  text = f.read()
  lines = text.split()
  train_img_files = []
  train_labels = []
  for l in lines:
    data = l.split(',')
    train_img_files.append(img_dir+'/'+data[0])
    train_labels.append(int(data[1]))


  img_train_array=[]
  
  img_train_array = []
  for filename,label in zip(train_img_files,train_labels):
      
    image= cv2.imread( filename, cv2.COLOR_BGR2RGB)
    image=cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
    image=np.array(image)
    image = image.astype('float32')
    image /= 255 
    img_train_array.append(image)
      

  img_list = data_folder+'/test.csv'

  f = open(img_list)
  text = f.read()
  lines = text.split()
  test_img_files = []
  test_labels = []
  for l in lines:
    data = l.split(',')
    test_img_files.append(img_dir+'/'+data[0])
    test_labels.append(int(data[1]))


  img_test_array=[]

  
  for filename,label in zip(test_img_files,test_labels):

    image= cv2.imread( filename, cv2.COLOR_BGR2RGB)
    
    image=cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
    image=np.array(image)
    image = image.astype('float32')
    image /= 255 
    img_test_array.append(image)


  return img_train_array, train_labels, img_test_array, test_labels
    
def transform_dataset(x_train,y_train,x_test,y_test):

    
    
  x_test=    np.array(x_test)
  x_test.reshape(-1, IMG_HEIGHT, IMG_WIDTH, 3)
  y_test= np.array(y_test)
  x_train = np.array(x_train)
  x_train.reshape(-1, IMG_HEIGHT, IMG_WIDTH, 3)

  y_train = np.array(y_train)
  x_train = np.expand_dims(x_train, -1)
  x_test = np.expand_dims(x_test, -1)
  y_test = keras.utils.to_categorical(y_test,len(emotion_class))
  y_train = keras.utils.to_categorical(y_train,len(emotion_class))
  return x_train,y_train,x_test,y_test


2021-09-09 15:05:39.291645: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


Version:  2.5.0
Eager mode:  True
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is available


2021-09-09 15:05:40.305960: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-09-09 15:05:40.307738: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-09-09 15:05:40.350841: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-09 15:05:40.351317: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2070 SUPER computeCapability: 7.5
coreClock: 1.77GHz coreCount: 40 deviceMemorySize: 7.76GiB deviceMemoryBandwidth: 417.29GiB/s
202

In [2]:
import neptune
neptune.init(project_qualified_name='cchantra/keras-tuner',api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI0M2NkYzQyZC01M2MzLTRhYjQtOTQ5Ny05NGY0NTU5MmU2NjUifQ==') # your credentials
  #load data set
x_train, y_train, x_test, y_test =load_dataset(data_dir,source_dir)

x_train, y_train, x_test, y_test = transform_dataset(x_train, y_train, x_test, y_test )



In [3]:

#Squeeze random
hp = HyperParameters()

import neptune


PROJECT = "squeezeSEAutoRandom_Tsinghua"+str(ratio)
neptune.create_experiment(PROJECT)


# This will override the `learning_rate` parameter with your
# own selection of choices

hp.Choice("learning_rate", values=[ 1e-3, 1e-4])



#fn_name = globals()["build_SqueezeNet_11_fixed"](hp)
# SqueezeNet 

mymodel = SqueezeNetSEAutoModel (classes=nb_classes, IMG_HEIGHT=IMG_HEIGHT,IMG_WIDTH=IMG_WIDTH)


tuner = RandomSearch(
  #build_SqueezeNet_11_fixed,
  #build_squeezenet_auto_model,
  #build_model,

  hypermodel=mymodel,
  hyperparameters = hp,
  objective=Objective("val_accuracy", direction="max"),
  max_trials=200,
  executions_per_trial=1,
  overwrite=True,
  directory="mytest_tsinghua_dir",
  project_name=PROJECT,
  #distribution_strategy=tf.distribute.MirroredStrategy(),
  logger = npt_utils.NeptuneLogger(),

)
tuner.search_space_summary()


   
 

https://app.neptune.ai/cchantra/keras-tuner/e/KER-103
[False, False]
['max', 'max']
[False, False]
Search space summary
Default search space size: 24
learning_rate (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.001, 0.0001], 'ordered': True}
compression (Fixed)
{'conditions': [], 'value': 1.0}
fire_module (Int)
{'default': 2, 'conditions': [], 'min_value': 1, 'max_value': 2, 'step': 1, 'sampling': None}
use_bypass0 (Boolean)
{'default': False, 'conditions': []}
use_bypass1 (Boolean)
{'default': False, 'conditions': []}
pooling0 (Choice)
{'default': 'max', 'conditions': [], 'values': ['max', 'avg'], 'ordered': False}
pooling1 (Choice)
{'default': 'max', 'conditions': [], 'values': ['max', 'avg'], 'ordered': False}
squeeze_ratio2 (Choice)
{'default': 32, 'conditions': [], 'values': [8, 16, 32], 'ordered': True}
SE_add2 (Boolean)
{'default': False, 'conditions': []}
SE_add3 (Boolean)
{'default': False, 'conditions': []}
squeeze_ratio4 (Choice)
{'default': 32, 'conditions': [],

2021-09-09 15:06:46.009855: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-09 15:06:46.010203: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2070 SUPER computeCapability: 7.5
coreClock: 1.77GHz coreCount: 40 deviceMemorySize: 7.76GiB deviceMemoryBandwidth: 417.29GiB/s
2021-09-09 15:06:46.010276: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-09 15:06:46.010589: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-09 15:06:46.010876: I tensorflow/core/

In [4]:
len(x_test)

175

In [5]:

""""lr_finder = LRFinder(min_lr=1e-5, 
                  max_lr=1e-3,
                  steps_per_epoch=np.ceil(EPOCH/batch_size),
                  epochs=3)"""
my_callbacks = [
  tf.keras.callbacks.EarlyStopping('val_accuracy', patience=5),
  #lr_finder,
  #tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
  #tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

tuner.search(x_train[:int(ratio*len(x_train))], y_train[:int(ratio*len(y_train))], epochs=EPOCH, callbacks = my_callbacks,validation_data=(x_test[:int(ratio*len(x_test))], y_test[:int(ratio*len(y_test))]))

models = tuner.get_best_models(num_models=10)



tuner.results_summary()
npt_utils.log_tuner_info(tuner)


#best_model = tuner.get_best_models(num_models=1)[0]
#best_model.evaluate(x_test, y_test)





Trial 200 Complete [00h 00m 07s]
val_accuracy: 0.1257142871618271

Best val_accuracy So Far: 0.508571445941925
Total elapsed time: 00h 24m 52s
INFO:tensorflow:Oracle triggered exit
[False, True]
['avg', 'avg']
[]
[False, False]
['max', 'avg']
[True]
[False]
['max']
[False, True]
[True, False]
['avg', 'avg']
[False]
[False, True]
['avg', 'max']
[False, True]
[True]
['max']
[True]
[False]
['max']
[]
[False, True]
['max', 'avg']
[True]
[True, False]
['avg', 'avg']
[False, False]
[True]
['max']
[]
Results summary
Results in mytest_tsinghua_dir/squeezeSEAutoRandom_Tsinghua1.0
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
learning_rate: 0.001
compression: 1.0
fire_module: 2
use_bypass0: False
use_bypass1: True
pooling0: avg
pooling1: avg
squeeze_ratio2: 8
SE_add2: True
SE_add3: True
squeeze_ratio4: 8
SE_add4: False
SE_add5: False
num_fire_2: 0
use_bypass_20: True
use_bypass_21: True
squeeze_ratio6: 32
SE_add6: True
SE_add7: False
squeez

In [6]:
i = 0
while i < len(models):
    print('model  %d size %d'%(i,hypermodel.maybe_compute_model_size(models[i])))
    i = i+1

model  0 size 145032
model  1 size 500880
model  2 size 467568
model  3 size 490264
model  4 size 1001720
model  5 size 161776
model  6 size 44984
model  7 size 565112
model  8 size 903964
model  9 size 42928


In [7]:
len(models)

10

In [10]:
#Squeeze Bayesian
hp = HyperParameters()

import neptune

PROJECT = "squeezeSEAutoBayesian_Tsinghua"+str(ratio)
neptune.create_experiment(PROJECT)


# This will override the `learning_rate` parameter with your
# own selection of choices

hp.Choice("learning_rate", values=[ 1e-3, 1e-4])



#fn_name = globals()["build_SqueezeNet_11_fixed"](hp)
# SqueezeNet 

mymodel = SqueezeNetSEAutoModel  (classes=nb_classes,IMG_HEIGHT=IMG_HEIGHT,IMG_WIDTH=IMG_WIDTH)


tuner = BayesianOptimization(
  #build_SqueezeNet_11_fixed,
  #build_squeezenet_auto_model,
  #build_model,

  hypermodel=mymodel,
  hyperparameters = hp,
  objective="val_accuracy",
  max_trials=100,
  executions_per_trial=1,
  overwrite=True,
  directory="mytest_tsinghua_dir",
  project_name= PROJECT,
  #distribution_strategy=tf.distribute.MirroredStrategy(),
  num_initial_points=2,
  alpha=0.0001,
  beta=2.6,
  seed=None,
  tune_new_entries=True,
  allow_new_entries=True,
  logger = npt_utils.NeptuneLogger(),

)
tuner.search_space_summary()


  

https://app.neptune.ai/cchantra/keras-tuner/e/KER-105
[False, False]
['max', 'max']
[False, False]
Search space summary
Default search space size: 24
learning_rate (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.001, 0.0001], 'ordered': True}
compression (Fixed)
{'conditions': [], 'value': 1.0}
fire_module (Int)
{'default': 2, 'conditions': [], 'min_value': 1, 'max_value': 2, 'step': 1, 'sampling': None}
use_bypass0 (Boolean)
{'default': False, 'conditions': []}
use_bypass1 (Boolean)
{'default': False, 'conditions': []}
pooling0 (Choice)
{'default': 'max', 'conditions': [], 'values': ['max', 'avg'], 'ordered': False}
pooling1 (Choice)
{'default': 'max', 'conditions': [], 'values': ['max', 'avg'], 'ordered': False}
squeeze_ratio2 (Choice)
{'default': 32, 'conditions': [], 'values': [8, 16, 32], 'ordered': True}
SE_add2 (Boolean)
{'default': False, 'conditions': []}
SE_add3 (Boolean)
{'default': False, 'conditions': []}
squeeze_ratio4 (Choice)
{'default': 32, 'conditions': [],

In [11]:
""""lr_finder = LRFinder(min_lr=1e-5, 
                  max_lr=1e-3,
                  steps_per_epoch=np.ceil(EPOCH/batch_size),
                  epochs=3)"""
my_callbacks = [
  tf.keras.callbacks.EarlyStopping('val_accuracy', patience=10),
  #lr_finder,
  #tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
  #tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

tuner.search(x_train[:int(ratio*len(x_train))], y_train[:int(ratio*len(y_train))], epochs=EPOCH, callbacks = my_callbacks,validation_data=(x_test[:int(ratio*len(x_test))], y_test[:int(ratio*len(y_test))]))

models = tuner.get_best_models(num_models=10)
tuner.results_summary()
npt_utils.log_tuner_info(tuner)

Trial 100 Complete [00h 00m 12s]
val_accuracy: 0.1257142871618271

Best val_accuracy So Far: 0.6514285802841187
Total elapsed time: 01h 16m 51s
INFO:tensorflow:Oracle triggered exit
[False, False]
['avg', 'avg']
[]
[False, False]
['avg', 'max']
[]
[False, False]
['avg', 'max']
[]
[False, False]
['avg', 'max']
[]
[False, False]
['avg', 'max']
[]
[False, False]
['avg', 'max']
[]
[False, False]
['avg', 'max']
[]
[False, False]
['avg', 'max']
[]
[False, False]
['avg', 'max']
[]
[False, False]
['avg', 'max']
[]
Results summary
Results in mytest_tsinghua_dir/squeezeSEAutoBayesian_Tsinghua1.0
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
learning_rate: 0.001
compression: 1.0
fire_module: 2
use_bypass0: False
use_bypass1: False
pooling0: avg
pooling1: avg
squeeze_ratio2: 8
SE_add2: False
SE_add3: False
squeeze_ratio4: 16
SE_add4: True
SE_add5: True
num_fire_2: 0
use_bypass_20: True
use_bypass_21: True
squeeze_ratio6: 8
SE_add6: False
SE_a

In [12]:


i = 0
while i < len(models):
    print('model  %d size %d'%(i,hypermodel.maybe_compute_model_size(models[i])))
    i = i+1

model  0 size 174344
model  1 size 174344
model  2 size 174344
model  3 size 174344
model  4 size 174344
model  5 size 174344
model  6 size 174344
model  7 size 174344
model  8 size 174344
model  9 size 174344


In [14]:
 
#SqueezeAuto Hyper
hp   = HyperParameters()


# This will override the `learning_rate` parameter with your
# own selection of choices
hp.Choice("learning_rate", values=[ 1e-3, 1e-4])


import neptune

PROJECT = "squeezeSEAutoHyperTsinghua"+str(ratio)
neptune.create_experiment(PROJECT)


#fn_name = globals()["build_SqueezeNet_11_fixed"](hp)
# SqueezeNet 

mymodel = SqueezeNetSEAutoModel(classes=nb_classes,IMG_HEIGHT=IMG_HEIGHT,IMG_WIDTH=IMG_WIDTH)



tuner = Hyperband(
  #build_SqueezeNet_11_fixed,
  #build_squeezenet_auto_model,
  #build_model,

  hypermodel=mymodel,
  hyperparameters = hp,
  objective="val_accuracy",
  #max_trials=20,
  executions_per_trial=1,
  overwrite=True,
  directory="mytest_tsinghua_dir",
  project_name=PROJECT,
  #distribution_strategy=tf.distribute.MirroredStrategy(),
  max_epochs=EPOCH,
  logger = npt_utils.NeptuneLogger(),

)
tuner.search_space_summary()


    


https://app.neptune.ai/cchantra/keras-tuner/e/KER-107
[False, False]
['max', 'max']
[False, False]
Search space summary
Default search space size: 24
learning_rate (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.001, 0.0001], 'ordered': True}
compression (Fixed)
{'conditions': [], 'value': 1.0}
fire_module (Int)
{'default': 2, 'conditions': [], 'min_value': 1, 'max_value': 2, 'step': 1, 'sampling': None}
use_bypass0 (Boolean)
{'default': False, 'conditions': []}
use_bypass1 (Boolean)
{'default': False, 'conditions': []}
pooling0 (Choice)
{'default': 'max', 'conditions': [], 'values': ['max', 'avg'], 'ordered': False}
pooling1 (Choice)
{'default': 'max', 'conditions': [], 'values': ['max', 'avg'], 'ordered': False}
squeeze_ratio2 (Choice)
{'default': 32, 'conditions': [], 'values': [8, 16, 32], 'ordered': True}
SE_add2 (Boolean)
{'default': False, 'conditions': []}
SE_add3 (Boolean)
{'default': False, 'conditions': []}
squeeze_ratio4 (Choice)
{'default': 32, 'conditions': [],

In [15]:

""""lr_finder = LRFinder(min_lr=1e-5, 
                  max_lr=1e-3,
                  steps_per_epoch=np.ceil(EPOCH/batch_size),
                  epochs=3)"""
my_callbacks = [
  tf.keras.callbacks.EarlyStopping('val_accuracy', patience=10),
  #lr_finder,
  #tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
  #tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

tuner.search(x_train[:int(ratio*len(x_train))], y_train[:int(ratio*len(y_train))], epochs=EPOCH, callbacks = my_callbacks,validation_data=(x_test[:int(ratio*len(x_test))], y_test[:int(ratio*len(y_test))]))

models = tuner.get_best_models(num_models=10)
tuner.results_summary()

Trial 254 Complete [00h 00m 10s]
val_accuracy: 0.1314285695552826

Best val_accuracy So Far: 0.668571412563324
Total elapsed time: 00h 42m 52s
INFO:tensorflow:Oracle triggered exit
[False, False]
['avg', 'max']
[False]
[False, False]
['max', 'max']
[]
[True]
['max']
[False]
[True]
['max']
[False]
[True]
['max']
[False]
[True]
['avg']
[False, False]
[False]
['max']
[False]
[True]
['avg']
[True, True]
[True]
['avg']
[False]
[False, False]
['max', 'max']
[False]
Results summary
Results in mytest_tsinghua_dir/squeezeSEAutoHyperTsinghua1.0
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
learning_rate: 0.001
compression: 1.0
fire_module: 2
use_bypass0: False
use_bypass1: False
pooling0: avg
pooling1: max
squeeze_ratio2: 32
SE_add2: True
SE_add3: False
squeeze_ratio4: 8
SE_add4: False
SE_add5: True
num_fire_2: 1
use_bypass_20: False
use_bypass_21: True
squeeze_ratio6: 8
SE_add6: True
SE_add7: True
squeeze_ratio8: 8
SE_add8: True
SE_add9: T

In [16]:
i = 0
while i < len(models):
    print('model  %d size %d'%(i,hypermodel.maybe_compute_model_size(models[i])))
    i = i+1

model  0 size 545356
model  1 size 183048
model  2 size 180864
model  3 size 192944
model  4 size 180864
model  5 size 358996
model  6 size 147200
model  7 size 441768
model  8 size 140792
model  9 size 522384


Unexpected error in ping thread.
Traceback (most recent call last):
  File "/home/chantana/anaconda3/envs/tf2.5/lib/python3.9/site-packages/urllib3/connection.py", line 169, in _new_conn
    conn = connection.create_connection(
  File "/home/chantana/anaconda3/envs/tf2.5/lib/python3.9/site-packages/urllib3/util/connection.py", line 73, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/home/chantana/anaconda3/envs/tf2.5/lib/python3.9/socket.py", line 953, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -3] Temporary failure in name resolution

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/chantana/anaconda3/envs/tf2.5/lib/python3.9/site-packages/urllib3/connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "/home/chantana/anaconda3/envs/tf2.5/lib/python3.9/sit